## 5.6 사용자-사용자 메모리 기반 방법 협조 필터링
- 메모리 기반 방법은 추천 시스템이 사용되기 전까지 데이터를 축적하고 계산을 수행하지 않음
  - 예측 계산 시 시간이 걸림
- 수행 과정은 아래와 같음
  1. 미리 얻은 평갓값을 사용해 사용자 사이의 유사도를 계산하고 추천받을 사용자와 기호 경향이 비슷한 사용자를 찾는다
  2. 기호 경향이 비슷한 사용자의 평갓값으로 추천받을 사용자의 미지의 아이템에 대한 예측 평갓값을 계산
  3. 예측 평갓값이 높은 아이템을 사용자에게 추천

- 우선 미리 얻은 사용자의 평갓값에 기반해 사용자 사이의 유사도를 계산
- 여기서는 피어슨 상관 계수 사용
- (src/umcf.py 참고)
![피어슨계수](./images/math-exp_1.png)

- 다음으로 피어슨 계수를 사용해 실제로 사용자 사이의 유사도 산출
- 평갓값을 예측할 대상 사용자와 그 외 사용자의 유사도를 산출
- 여기서는 유사도가 0보다 큰 경우 비슷한 사용자로 간주하고 비슷한 사용자의 ID, 유사도, 평갓값 평균을 각각 변수 similar_users, similarities, avgs에 저장

- 그 다음으로 예측 대상 사용자와 기호 경향이 비슷한 사용자의 평갓값을 사용해 미지의 아이템에 대한 예측 평갓값을 계산
- 대량 예측 수행시에는 상당한 시간이 소요되므로 평가하지 않는 모든 영화에 대한 평갓값을 예측해야 하는 순위 형식의 추천 리스트는 작성하지 않음
  - 테스트용 데이터에 존재하는 사용자와 영화의 조합에 대한 평갓값만 예측 계산하여 RMSE를 통한 성능 평가만 수행
- 예측 평갓값 계산에는 '사용자별 평균 평갓값에서 해당 아이템에 대한 평가가 얼마나 높은 평ㄹ가인가라는 상대적인 평갓값에 주목해 해당 값의 가중 평균을 얻는다'는 방법을 사용
- 이 계산 방법(과 몇 가지 패턴)이 궁금하면 부록B 참고
![가중 평균](./images/math-exp_2.png)

- 이를 통해 얻은 미평가 영화에 대한 예측 평갓값 중 해당 값이 높은 것을 사용자에게 추천
- 평가 결과는 RMSE=0.956
- src/umcf.py 함수는 파이썬 추천 시스템의 Surprise 라이브러리를 사용해서 구현
- 아래는 실행 코드

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# 부모 폴더의 경로를 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [4]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='./data/ml-10M100K/')
movielens = data_loader.load()

In [ ]:
# ! pip install scikit-surprise

In [5]:
from src.umcf import UMCFRecommender

ModuleNotFoundError: No module named 'surprise'

In [ ]:
recommender = UMCFRecommender()
recommend_result = recommender.recommend(movielens, is_naive=True)

In [ ]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

- 평가 결과는 RMSE=0.962, Precision@K=0.002, Recall@K=0.005
- 이것은 평가 수는 적지만 평갓값이 높은 아이템의 영향도가 크기 때문이며 일정 수 이상 평가된 아이템만 남기면 정확도를 높일 수 있음

## 5.7 회귀 모델
- 미지의 아이템에 대한 평갓값을 회귀 문제로 예측
- MovieLens의 예에서 평갓값이 0.5~5 간격의 값을 가지므로 이를 회귀 모델에서 예측
- 여기서는 랜덤 포레스트를 사용 (구현은 sklearn.ensenble)

- 코드 설명
  - 특징량(feature)를 작성
  - 여기서의 피처는 사용자별 평갓값의 최솟값, 최댓값, 평균값 및 영화별 최대, 최소, 평균값
  - 학습용 데이터 전체 평균 평가값으로 채워둠
  - 장르 정보도 추가 사용, 어떤 영화가 특정 장르에 속하는지 나타내는 boolean 피처 추가
- 테스트용 데이터에 대한 예측 평갓값과 순위 형식의 추천 리스트 평가 결과는 아래와 같음
  - RMSE=0.988, Precision@K=0.000, Recall@K=0.001
- 평가 수는 적지만 평가가 높은 아이템에 대해 높은 평갓값이 예측되었고 정확도가 떨어졌음
  - 평가 수의 임곗값을 설정하면 이 현상을 피할 수 있음

### 5.8 행렬 분해
### 5.8.1 행렬 분해 개요
- 추천 시스템에서의 행렬 분해는 넓은 의미에서 평갓값 행렬을 저차원의 사용자 인자 행렬과 아이템 인자 행렬로 분해하는 것을 나타냄
- 사용자와 아이템을 100차원 정도의 저차원 벡터로 표현하고 그 벡터의 내적값을 사용자와 아이템의 상성
- 이번 절에서는 SVD, NMF, MF, IMF, BPR, FM 행렬 분해 방법을 순서대로 설명
- 행렬 분해 방법에서는 '결손값 취급', '평가값이 명시적인지 암묵적인지'라는 관점이 매우 중요
- MovieLens 데이터는 사용자가 명시적으로 평가한 것이어서 그 품질이 높음
  ![행렬 분해 개요](./images/img_5-12.png)

- 모든 사용자 X 모든 아이템 조합은 극히 일부 값만 존재 (모든 사용자가 모든 아이템 평가 X)
- 평가값 행렬이 주어졌을 때 사용자와 아이템을 저차원 벡터로 표현할 수 있음
  - MovieLens 데이터는 사용자 1000명, 아이템이 6673개임
  - 1명의 사용자는 아이템 수의 6673 차원의 벡터로 표현
  - 저차원 벡터란 1명의 사용자가 1개의 작품을 1차원으로 표현하는 것이 아니고 다차원에 많은 영화로 표현
  - 아래는 사용자/아이템의 잠재 벡터 공간 예시
    ![잠재 벡터](./images/img_5-14.png)

  - 위처럼 정보를 압축해서 영화와 사용자를 저차원의 벡터로 표현하고 공간 안에서 사용자 상성을 측정하는 것이 행렬 분해 핵심

### 5.8.2 특잇값 분해
- 특이값 분해(Singular Value Decomposition, SVD)로 차원을 줄일 수 있음
  - 관련 수식은 추후 첨부
  - 코드도 추후 첨부
- 이 방법은 결손값을 0으로 채우기 때문에 추천 성능이 나쁘다고 알려져있음
  - 평가값 결손은 사용자가 평가를 하지 않은 것이며 대부분 값이 0이 되기에 예측 평가값도 0에 가까워져 RMSE 지표도 나빠짐
  - 0이 아닌 평균 값을 대입하기도 함
- SVD에는 잠재 인자 수라는 중요 파라미터가 있으며 이를 변화시키면 정확도가 바뀜
  - 잠재 인자 수가 높을수록 원래 행렬을 복구할 때 충분한 표현력을 가지므로 예측 정확도가 높아짐
  - 너무 높으면 과적합 가능성이 있음
  - 일반적으로 수십~수백 정도로 설정

### 5.8.3 비음수 행렬 분해
- 이전 절의 SVD는 행렬 분해 후 행렬에 대해 음수값을 취하는 경우가 있지만 NMF는 행렬 분해 시 사용자 X 아이템 각 벡터의 요소가 0 이상 되는 제약을 추가
- NMF 역시 결손값을 0으로 채워 적용하는 경우가 많음
  - 수식 추가 예정
  - 코드 추가 예정
- sklearn 라이브러리에 P와 Q의 비음수 행렬을 얻을 수 있으며 이것으로 예측을 수행
- 정확도 관점에서 5.8.4 이후의 알고리즘 사용 권장 

### 5.8.4 명시적인 평갓값에 대한 행렬 분해
- 행렬 분해(matrix factorization, MF)는 스파크나 빅쿼리 등에도 구현
- 최적화 문제로 풀이함(수식 추가 예정)
- MF는 ALS(alterniating least square), SGD(stochastic gradient descent)를 사용하는 방법이 제안됨
  - ALS : 목적 함수가 최소화되도록 사용자 인자 행렬과 아이템 인자 행렬을 교대로 최적화
  - SGD : 입력 데이터를 샘플링해 그 데이터 점에 대한 사용자 인자 행렬과 아이템 인자 행렬의 기울기를 계산하고 p와 q를 기울기 방향으로 업데이트
  - 이 식의 총합 계산 부분에서 관측된 평가값만 사용해 사용자와 아이템 벡터를 구함
  - 이 벡터로 사용자의 내적을 계산해 추천 아이템 추출
  - 코드 추가 예정
- 행렬 분해에서는 잠재 인자 수와 에포크 수, 정규화 파라미터 조정이 중요
- 이 파라미터의 적절한 값은 그리드 서치나 베이즈 최적화 등으로 찾아야 함

### 5.8.5 암묵적인 평갓값에 대한 행렬 분해
- 유저 행동 로그 등을 암묵적 평가값이라고 표현
- 암묵적 평가값에 대해 행렬 분해(Implicit Matrix Factorization, IMF)가 제안됨
  - 수식 추가 예정
- 반복해서 열람할수록 선호도에 대한 신뢰도가 높아짐
- MF의 경우 관측된 평가값으로 총합을 얻는데 비해 IMF 식에서는 사용자와 아이템의 모든 조합으로 총합을 구함
- 암묵적 평가에 부정적 예시가 없는데 기인해 관측되지 않은 것은 부정적 예시로 간주
- 실무에서 행렬 분해를 사용할 때는 스파크, Google ML 등의 라이브러리나 서비스에 구현되어 있는 행렬분해가 명시적인지 암묵적인지 확인후 적절히 사용해야 함
- 코드 추가 예정
- 결과에 대해 데이터 희박한 정도, 평가값 분포의 편향등을 확인해서 적합한 설계를 해야함

### 5.8.6 BPR
- 암묵적 평가의 다른 유명한 방법으로 BPR(Bayesian Personalized Ranking)이 있음
- 암묵적으로 평가한 아이템 i의 경우 관측되지 않은 아이템 j에 비해 사용자가 선호한다는 가정으로 계산
  - 수식 추가 예정
- 관측되지 않은 아이템의 샘플링 방법을 잘 선택해야 함
  - 출현 횟수에 따른 샘플링
  - 클릭했지만 구매하지 않은 아이템으로 샘플링 등
- 수식 추가 예정

### 5.8.7 FM
- 이 방법은 콜드 스타트 문제에도 대응할 수 있다는 장점이 있음
- FM(Factorization Machines)은 1개의 평가에 대한 정보를 1행으로 표시
  - 행렬은 평가값X피처의 수가 됨
  - 피처는 아래의 3가지 정보가 됨
    - 사용자 ID를 원-핫-인코딩한 것
    - 아이템 ID를 원-핫-인코딩한 것
    - 사용자와 아이템의 속성 정보 등의 보조 정보를 연결한 것(사용자의 나이, 아이템 추가 후 경과 일수 등)
  - 이를 사용해 평가값 y를 예측
    ![FM의 데이터 구조](./images/img_5-16.png)

- FM의 장점은 피처사이의 조합도 고려할 수 있다는 것
  - 수식 추가 예정
  - 코드 추가 예정
  